# Стохатика. Лабораторная работа по 3 модели

### Команда __Название команды__:
- Меркулов Александр
- Калугин Евгений
- Фомин Дмитрий
- Чукаев Иван

## Описание модели

На вход $n$-канальной СМО поступает простейший поток заявок с интенсивностью $\lambda$.

Интенсивность простейшего потока обслуживания каждого канала $\mu$.

Если заявка застаёт все каналы свободными, она принимается на обслуживание и обслуживается любым одним из $n$-каналов.

Если заявка застаёт свободным хотя бы один канал, то она принимается на обслуживание любым из свободных каналов и обслуживается до конца.

Если заявка застаёт все каналы занятыми, то она получает отказ (покидает систему не обслуженной).

После окончания обслуживания одной заявки освобождается один канал.

Состояние рассмотренной системы будем связывать с числом заявок, находящихся в системе.

### Граф состояний

![image.png](./images/graph.jpg)

Рассмотрим множество состояний системы:

$S_0$ — в системе нет ни одной заявки, все каналы свободны;

$S_1$ — в системе имеется одна заявка, она обслуживается одним каналом;

$S_2$ — в системе имеется две заявки, они обслуживаются двумя каналами;

…

$S_k$ — в системе имеется $k$-заявок, они обслуживаются $k$-каналами;

$S_{k+1}$ — в системе имеется $(k+1)$-заявок, они обслуживаются $(k+1)$-каналами;

…

$S_{n-1}$ — в системе имеется $(n-1)$-заявок, они обслуживаются $(n-1)$-каналами;

$S_n$ — в системе имеется $n$-заявок, они обслуживаются $n$-каналами.

## Постановка задачи

В городе имеется n станций техобслуживания.

Сервисы обслуживает $\lambda$ машин в день, интенсивность входного потока заявок - $\mu$ машин в день. 

После окончания обслуживания один сервис освобождается. 

Если вновь прибывшая машина застаёт свободным хотя бы один сервис, то она принимается на обслуживание одним из свободных сервисов. 

Если нет свободных сервисов в городе, то машина уезжает на обслуживание в другой город.

## Мат. модель.

0\) $\;\mu\Pi_1=\lambda\Pi_0$

1\) $\;\lambda\Pi_0+2\mu\Pi_2=(\lambda+\mu)\Pi_1$

2\) $\;\lambda\Pi_1+3\mu\Pi_3=(\lambda+2\mu)\Pi_2$

…

k) $\;\lambda\Pi_{k-1}+(k+1)\mu\Pi_{k+1}=(\lambda+k\mu)\Pi_k$

…

n) $\;\lambda\Pi_{n-1}= n\mu\Pi_n$ <br><br><hr><br>

1\) $\;\Pi_1=\frac{\lambda}{\mu}\Pi_0$<br><br>

2\) $\;2\mu\Pi_2=(\lambda+\mu)\Pi_1-\lambda\Pi_0$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;2\mu\Pi_2=(\lambda+\mu)\frac{\lambda}{\mu}\Pi_0-\lambda\Pi_0$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;2\mu\Pi_2=\frac{\lambda^2+\lambda\mu-\lambda\mu}{\mu}\Pi_0$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;\Pi_2=\frac{1}{2}(\frac{\lambda}{\mu})^2\Pi_0$ <br>&nbsp;&nbsp;&nbsp;

3\) $\;3\mu\Pi_3=(\lambda+2\mu)\Pi_2-\lambda\Pi_1$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;3\mu\Pi_3=(\lambda+2\mu)(\frac{1}{2})(\frac{\lambda}{\mu})^2\Pi_0-\lambda\frac{\lambda}{\mu}\Pi_0$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;3\mu\Pi_3=(\frac{\lambda}{\mu})^2(\frac{1}{2}(\lambda+2\mu)\Pi_0-\mu\Pi_0)$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;3\mu\Pi_3=(\frac{\lambda}{\mu})^2\cdot\frac{\lambda}{2}\Pi_0$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;\Pi_3=(\frac{\lambda}{\mu})^3\cdot\frac{1}{2\cdot3}\Pi_0$ <br><br>

k)  $\;\Pi_k=(\frac{\lambda}{\mu})^k\cdot\frac{1}{k!}\Pi_0$